In [ ]:
!pip install eccodes herbie-data ecmwflibs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 MB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.5/400.5 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 99.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.0/294.0 kB 33.5 MB/s eta 0:00:00
  Created wheel for eccodes: filename=eccodes-1.6.1-py3-none-any.whl size=41000 sha256=a1f1f8e57da6fe89e6af71d6d6051fcd4de7b19bf89ebec8264567e4cef0496a
  Stored in directory: /root/.cache/pip/wheels/10/1f/ee/380d51d2c6c97c7677f918c5e45478c9cc23122abf5a273b2c
  Created w

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from herbie import FastHerbie
import pandas as pd

 ╭─────────────────────────────────────────────────╮
 │ I'm building Herbie's default config file.      │
 ╰╥────────────────────────────────────────────────╯
 👷🏻‍♂️
 ╭─────────────────────────────────────────────────╮
 │ You're ready to go.                             │
 │ You may edit the config file here:              │
 │ /root/.config/herbie/config.toml                │
 ╰╥────────────────────────────────────────────────╯
 👷🏻‍♂️


/usr/local/lib/python3.10/dist-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.30.0
  warnings.warn(


In [ ]:
config_string = """[default]
model = "hrrr"
fxx = 0
save_dir = "/root/data"
overwrite = false
verbose = true
priority = ['aws','google','azure']
"""

with open('/root/.config/herbie/config.toml', 'w') as f:
  f.write(config_string)


In [ ]:
!cat ${HOME}/.config/herbie/config.toml

[default]
model = "hrrr"
fxx = 0
save_dir = "/root/data"
overwrite = false
verbose = true
priority = ['aws','google','azure']


# Get the date roll

In [ ]:
from datetime import datetime, timedelta

def print_dates_between(start_date, end_date, time_suffix):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")

    current = start
    while current <= end:
        yield(current.strftime("%Y-%m-%d") + " " + time_suffix)
        current += timedelta(days=1)

# Logging Helper Function

In [ ]:
def log_download_successful(date):
  with open('downlog.txt', 'a') as f:
    f.write(date+'\n')

In [ ]:
def check_date_in_log(date):
  try:
    with open('downlog.txt', 'r') as f:
      for d in f:
        if date in d.strip():
          return True
  except:
    print("Maybe the log file does not exist. Proceed for now.")
    return False

  return False

# FastHerbie Downloader Script

In [ ]:
def download_vil(from_datetime, to_datetime):
  # Verify if the date has been downloaded
  if check_date_in_log(from_datetime):
    print('Skipped {} because it is in the log '.format(from_datetime))
    return

  # Begin downloading with slicing of 30 minutes
  DATES = pd.date_range(
    start=from_datetime, # begin date
    end=to_datetime, # end date
    freq="30min", # frequency: 30 minutes
  )

  roll_list = FastHerbie(DATES, model="hrrr", priority=['aws','google','azure'])

  # Download the VIL field
  roll_list.download("VIL")

  # Log download successful
  log_download_successful(from_datetime)

# Download Entry

In [ ]:
# Get the roll
from_datetime = list(print_dates_between("2018-01-01", "2019-12-31", "00:00"))
to_datetime = list(print_dates_between("2018-01-01", "2019-12-31", "23:45"))
print('Roll call created')

Roll call created


In [ ]:
print(from_datetime)
print(to_datetime)
print('There are {} days'.format(len(from_datetime)))

['2018-01-01 00:00', '2018-01-02 00:00', '2018-01-03 00:00', '2018-01-04 00:00', '2018-01-05 00:00', '2018-01-06 00:00', '2018-01-07 00:00', '2018-01-08 00:00', '2018-01-09 00:00', '2018-01-10 00:00', '2018-01-11 00:00', '2018-01-12 00:00', '2018-01-13 00:00', '2018-01-14 00:00', '2018-01-15 00:00', '2018-01-16 00:00', '2018-01-17 00:00', '2018-01-18 00:00', '2018-01-19 00:00', '2018-01-20 00:00', '2018-01-21 00:00', '2018-01-22 00:00', '2018-01-23 00:00', '2018-01-24 00:00', '2018-01-25 00:00', '2018-01-26 00:00', '2018-01-27 00:00', '2018-01-28 00:00', '2018-01-29 00:00', '2018-01-30 00:00', '2018-01-31 00:00', '2018-02-01 00:00', '2018-02-02 00:00', '2018-02-03 00:00', '2018-02-04 00:00', '2018-02-05 00:00', '2018-02-06 00:00', '2018-02-07 00:00', '2018-02-08 00:00', '2018-02-09 00:00', '2018-02-10 00:00', '2018-02-11 00:00', '2018-02-12 00:00', '2018-02-13 00:00', '2018-02-14 00:00', '2018-02-15 00:00', '2018-02-16 00:00', '2018-02-17 00:00', '2018-02-18 00:00', '2018-02-19 00:00',

In [ ]:
# DOWNLOAD SCRIPT

last_processed_datetime = ''
try:
  for i, begin_datetime in enumerate(from_datetime):
    end_datetime = to_datetime[i]
    last_processed_datetime = begin_datetime
    download_vil(begin_datetime, end_datetime)

except:
  print("An error occurred with datetime {}".format(last_processed_datetime))

Maybe the log file does not exist. Proceed for now.
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180101]


👨🏻‍🏭 Created directory: [/root/data/hrrr/20180102]


👨🏻‍🏭 Created directory: [/root/data/hrrr/20180103]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180104]


👨🏻‍🏭 Created directory: [/root/data/hrrr/20180105]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180106]


👨🏻‍🏭 Created directory: [/root/data/hrrr/20180107]


...

👨🏻‍🏭 Created directory: [/root/data/hrrr/20180914]


👨🏻‍🏭 Created directory: [/root/data/hrrr/20180915]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180916]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180917]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180918]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180919]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180920]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180921]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180922]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180923]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180924]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180925]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180926]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180927]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180928]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180929]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20180930]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20181001]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20181002]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20181003]
👨🏻‍🏭 Created directory: [/root/

👨🏻‍🏭 Created directory: [/root/data/hrrr/20190310]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190311]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190312]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190313]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190314]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190315]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190316]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190317]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190318]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190319]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190320]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190321]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190322]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190323]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190324]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190325]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190326]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190327]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190328]
👨🏻‍🏭 Created directory: [/root/

👨🏻‍🏭 Created directory: [/root/data/hrrr/20190504]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190505]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190506]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190507]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190508]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190509]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190510]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190511]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190512]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190513]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190514]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190515]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190516]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190517]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190518]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190519]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190520]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190521]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20190522]
👨🏻‍🏭 Created directory: [/root/

👨🏻‍🏭 Created directory: [/root/data/hrrr/20191122]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191123]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191124]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191125]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191126]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191127]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191128]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191129]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191130]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191201]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191202]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191203]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191204]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191205]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191206]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191207]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191208]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191209]
👨🏻‍🏭 Created directory: [/root/data/hrrr/20191210]
👨🏻‍🏭 Created directory: [/root/

# Copy to Google Drive

In [ ]:
!tar -czvf hrrr18_19.tgz /root/data/hrrr

Streaming output truncated to the last 5000 lines.
/root/data/hrrr/20191016/subset_03efba75__hrrr.t02z.wrfsfcf00.grib2
/root/data/hrrr/20191016/subset_ceefba75__hrrr.t06z.wrfsfcf00.grib2
/root/data/hrrr/20191016/subset_35efba75__hrrr.t01z.wrfsfcf00.grib2
/root/data/hrrr/20191016/subset_00efba75__hrrr.t13z.wrfsfcf00.grib2
/root/data/hrrr/20191016/subset_ccefba75__hrrr.t03z.wrfsfcf00.grib2
/root/data/hrrr/20191016/subset_d6efba75__hrrr.t07z.wrfsfcf00.grib2
/root/data/hrrr/20191016/subset_09efba75__hrrr.t05z.wrfsfcf00.grib2
/root/data/hrrr/20191016/subset_04efba75__hrrr.t10z.wrfsfcf00.grib2
/root/data/hrrr/20191016/subset_c6efba75__hrrr.t11z.wrfsfcf00.grib2
/root/data/hrrr/20191016/subset_38efba75__hrrr.t02z.wrfsfcf00.grib2
/root/data/hrrr/20191016/subset_2eefba75__hrrr.t06z.wrfsfcf00.grib2
/root/data/hrrr/20191016/subset_10efba75__hrrr.t04z.wrfsfcf00.grib2
/root/data/hrrr/20191016/subset_e1efba75__hrrr.t17z.wrfsfcf00.grib2
/root/data/hrrr/20191016/subset_31efba75__hrrr.t15z.wrfsfcf00.gri

In [ ]:
!cp hrrr18_19.tgz /content/drive/MyDrive/hrrr18_19.tgz

# Danger: Deleting data scripts

Deleting local data

In [ ]:
#!rm -rf /root/data

In [ ]:
#!rm -rf downlog.txt

Deleting Drive's data: DO NOT EXECUTE

In [ ]:
#!rm -rf hrrr0216.tgz

In [ ]:
#!rm -rf /content/drive/MyDrive/hrrr.tgz